#   Brain Tumor Classification using InceptionV3

##   Import Necessary Libraries


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os

## Plotting Function for Confusion Matrix

In [ ]:
def plot_confusion_mat(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
    import matplotlib.pyplot as plat
    import itertools

    valid = np.trace(cm) / np.sum(cm).astype('float')
    isclass = int(1 - valid)

    if cmap is None:
        cmap = plat.get_cmap('Blues')

    plat.figure(figsize=(8, 6))
    plat.imshow(cm, interpolation='nearest', cmap=cmap)
    plat.title(title)
    plat.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plat.xticks(tick_marks, target_names, rotation=45)
        plat.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plat.text(j, i, "{:0.4f}".format(cm[i, j]),
                      horizontalalignment="center",
                      color="white" if cm[i, j] > thresh else "black")
        else:
            plat.text(j, i, "{:,}".format(cm[i, j]),
                      horizontalalignment="center",
                      color="white" if cm[i, j] > thresh else "black")

    plat.tight_layout()
    plat.ylabel('True label')
    plat.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, isclass))
    plat.show()

## Data Augmentation

In [ ]:
generator_train = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
generator_test = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

## Creating the Train and Test Data

In [ ]:
train = generator_train.flow_from_directory('data/Training',
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode="categorical",
                                            color_mode='rgb')

test = generator_test.flow_from_directory('data/Testing',
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode="categorical",
                                          color_mode='rgb')

## Building the InceptionV3 Model


In [ ]:
inceptionV3_weight_path = 'data/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
inceptionV3_base_model = tf.keras.applications.InceptionV3(include_top=False, weights=inceptionV3_weight_path, input_shape=(224, 224, 3))

NUM_CLASSES = 4

inceptionV3 = ()
inceptionV3.add(inceptionV3_base_model)
inceptionV3.add(Flatten())
inceptionV3.add(Dropout(0.5))
inceptionV3.add(Dense(NUM_CLASSES, activation='softmax'))

inceptionV3.layers[0].trainable = False

inceptionV3.compile(loss='categorical_crossentropy',
                    optimizer=RMSprop(learning_rate=1e-4),
                    metrics=['accuracy'])

inceptionV3.summary()

## Callbacks

In [ ]:
if not os.path.exists('models'):
    os.makedirs('models')

model_es = EarlyStopping(monitor='loss', min_delta=1e-11, patience=12, verbose=1)
model_rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6, verbose=1)
model_mcp = ModelCheckpoint(filepath='models/inceptionV3_model_weights.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


## Training the InceptionV3 Model

In [ ]:
history = inceptionV3.fit(train,
                          steps_per_epoch=len(train),
                          epochs=30,
                          validation_data=test,
                          validation_steps=len(test),
                          callbacks=[model_es, model_rlr, model_mcp])

## Evaluate the InceptionV3 Model


In [ ]:
inceptionV3.evaluate(test)

## Plot InceptionV3 Model Performance

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(history.epoch) + 1)

plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('InceptionV3 Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('InceptionV3 Model Loss')

plt.tight_layout()
plt.show()

## Validate on Test Set for InceptionV3 Model

In [ ]:
X_test, y_test = next(test)
predictions = inceptionV3.predict(X_test)
y_test = np.argmax(y_test, axis=1)
pred = np.argmax(predictions, axis=1)

accuracy = accuracy_score(y_test, pred)
print('Test Accuracy = %.2f' % accuracy)

confusion_mtx = confusion_matrix(y_test, pred)
print(confusion_mtx)
plot_confusion_mat(confusion_mtx, ['glioma', 'meningioma', 'notumor', 'pituitary'], normalize=False)